<a href="https://colab.research.google.com/github/prp1519/prp1519/blob/main/model_gpu_test_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
import pandas as pd
import re
import torch
# import torch_xla
# import torch_xla.core.xla_model as xm
from torch.utils.data import Dataset, TensorDataset, DataLoader, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoTokenizer, DataCollatorWithPadding
# from keras.preprocessing.sequence import pad_sequences
import pickle
import os
from transformers import BertTokenizer
import numpy as np

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
train_df = pd.read_json("csvjson.json")
val_df   = pd.read_json("test1.json")

In [ ]:
train_df = train_df.dropna()
val_df = val_df.dropna()

In [ ]:
train_df

,,Premise,Hypothesis,uid,label
0,0,Yasin Malik (born 1966) is a Kashmiri separati...,This is example of Crime domain,bf628cf5-6948-4595-a6d0-daf99beb47b9,entailment
1,1,He was the Chairman of Jammu Kashmir Liberatio...,This is example of Politics domain,e66af435-eef1-491b-af2a-4825d54611e1,entailment
2,2,"Malik states that, as a young boy, he had witn...",This is example of Crime domain,51941f10-59f7-420a-b260-38d10f225c9c,entailment
3,3,His group was involved in attempting to disrup...,This is example of Politics domain,73520f08-c78c-4c98-bf72-2df71b1e83d2,entailment
4,4,"After getting released in 1986, the Tala Party...",This is example of Politics domain,28b13fa5-6291-4391-b5bb-83c43f1f1fe6,entailment
...,...,...,...,...,...
361,361,"Later, the closeness grew so much that the PML...",This is example of Politics domain,356a7080-19e9-4abe-9490-7cbb35e774f6,entailment
362,362,It may be recalled that Hafiz Abdul Karim was ...,This is example of Politics domain,fbcf0a68-1447-478f-8b61-cb60bbb297ea,entailment
363,363,He further said that Sajid Mir or the Central ...,This is example of Politics domain,8f3f8a0a-2e61-403d-ac08-b7d44b48efaa,entailment
364,364,Sajid Mir continued to be the talk of Pakistan...,This is example of Politics domain,05a73d2e-726b-4450-973c-c814ad0ec5fc,entailment


In [ ]:
val_df

,,Premise,Hypothesis,uid,label
0,0,Yasin Malik (born 1966) is a Kashmiri separati...,This is example of Crime domain,bf628cf5-6948-4595-a6d0-daf99beb47b9,entailment
1,1,He was the Chairman of Jammu Kashmir Liberatio...,This is example of Politics domain,e66af435-eef1-491b-af2a-4825d54611e1,entailment
2,2,"Malik states that, as a young boy, he had witn...",This is example of Crime domain,51941f10-59f7-420a-b260-38d10f225c9c,entailment
3,3,His group was involved in attempting to disrup...,This is example of Politics domain,73520f08-c78c-4c98-bf72-2df71b1e83d2,entailment
4,4,"After getting released in 1986, the Tala Party...",This is example of Politics domain,28b13fa5-6291-4391-b5bb-83c43f1f1fe6,entailment
5,5,In the run up to the Legislative Assembly elec...,This is example of Politics domain,3dd6c3da-0768-46db-947e-fe3583c20a48,entailment
6,6,"According to another Jamaat member, the ISL wa...",This is example of Politics domain,8ea35491-9522-4f15-b6bf-266deb697656,entailment
7,7,Yusuf Shah as well as Yasin Malik were arreste...,This is example of Crime domain,fec0d6c1-0706-42bf-969f-d994943ee6ed,entailment
8,8,The police refused to listen to any complaint....,This is example of Politics domain,002e30ba-5d28-45dc-bc78-b733d8bb5bbb,entailment
9,9,"Malik disagrees. ""Let me clear it, rigging in ...",This is example of Crime domain,f7066b71-3531-4dbc-9194-e68f534c215f,entailment


In [ ]:
train_df['Premise'] = train_df['Premise'].astype(str)
train_df['Hypothesis'] = train_df['Hypothesis'].astype(str)

In [ ]:
val_df['Premise'] = val_df['Premise'].astype(str)
val_df['Hypothesis'] = val_df['Hypothesis'].astype(str)

In [ ]:
train_df = train_df[(train_df['Premise'].str.split().str.len() > 0) & (train_df['Hypothesis'].str.split().str.len() > 0)]
val_df = val_df[(val_df['Premise'].str.split().str.len() > 0) & (val_df['Hypothesis'].str.split().str.len() > 0)]

In [ ]:
train_df

,,Premise,Hypothesis,uid,label
0,0,Yasin Malik (born 1966) is a Kashmiri separati...,This is example of Crime domain,bf628cf5-6948-4595-a6d0-daf99beb47b9,entailment
1,1,He was the Chairman of Jammu Kashmir Liberatio...,This is example of Politics domain,e66af435-eef1-491b-af2a-4825d54611e1,entailment
2,2,"Malik states that, as a young boy, he had witn...",This is example of Crime domain,51941f10-59f7-420a-b260-38d10f225c9c,entailment
3,3,His group was involved in attempting to disrup...,This is example of Politics domain,73520f08-c78c-4c98-bf72-2df71b1e83d2,entailment
4,4,"After getting released in 1986, the Tala Party...",This is example of Politics domain,28b13fa5-6291-4391-b5bb-83c43f1f1fe6,entailment
...,...,...,...,...,...
361,361,"Later, the closeness grew so much that the PML...",This is example of Politics domain,356a7080-19e9-4abe-9490-7cbb35e774f6,entailment
362,362,It may be recalled that Hafiz Abdul Karim was ...,This is example of Politics domain,fbcf0a68-1447-478f-8b61-cb60bbb297ea,entailment
363,363,He further said that Sajid Mir or the Central ...,This is example of Politics domain,8f3f8a0a-2e61-403d-ac08-b7d44b48efaa,entailment
364,364,Sajid Mir continued to be the talk of Pakistan...,This is example of Politics domain,05a73d2e-726b-4450-973c-c814ad0ec5fc,entailment


In [ ]:
val_df

,,Premise,Hypothesis,uid,label
0,0,Yasin Malik (born 1966) is a Kashmiri separati...,This is example of Crime domain,bf628cf5-6948-4595-a6d0-daf99beb47b9,entailment
1,1,He was the Chairman of Jammu Kashmir Liberatio...,This is example of Politics domain,e66af435-eef1-491b-af2a-4825d54611e1,entailment
2,2,"Malik states that, as a young boy, he had witn...",This is example of Crime domain,51941f10-59f7-420a-b260-38d10f225c9c,entailment
3,3,His group was involved in attempting to disrup...,This is example of Politics domain,73520f08-c78c-4c98-bf72-2df71b1e83d2,entailment
4,4,"After getting released in 1986, the Tala Party...",This is example of Politics domain,28b13fa5-6291-4391-b5bb-83c43f1f1fe6,entailment
5,5,In the run up to the Legislative Assembly elec...,This is example of Politics domain,3dd6c3da-0768-46db-947e-fe3583c20a48,entailment
6,6,"According to another Jamaat member, the ISL wa...",This is example of Politics domain,8ea35491-9522-4f15-b6bf-266deb697656,entailment
7,7,Yusuf Shah as well as Yasin Malik were arreste...,This is example of Crime domain,fec0d6c1-0706-42bf-969f-d994943ee6ed,entailment
8,8,The police refused to listen to any complaint....,This is example of Politics domain,002e30ba-5d28-45dc-bc78-b733d8bb5bbb,entailment
9,9,"Malik disagrees. ""Let me clear it, rigging in ...",This is example of Crime domain,f7066b71-3531-4dbc-9194-e68f534c215f,entailment


In [ ]:
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import pickle
import os
from transformers import BertTokenizer

class MNLIbertbaseduncased(Dataset):

  def __init__(self, train_df, val_df):
    self.label_dict = {'entailment':0,
          'contradiction':1,
          'neutral':2}

    self.train_df = train_df
    self.val_df = val_df

    self.base_path = '/content/'
    self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    self.train_data = None
    self.val_data = None
    self.init_data()

  def init_data(self):
    self.train_data = self.load_data(self.train_df)
    self.val_data = self.load_data(self.val_df)

  def load_data(self, df):
    MAX_LEN = 512
    token_ids = []
    mask_ids = []
    seg_ids = []
    y = []

    premise_list = df['Premise'].to_list()
    hypothesis_list = df['Hypothesis'].to_list()
    label_list = df['label'].to_list()

    for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
      premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
      hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
      pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
      premise_len = len(premise_id)
      hypothesis_len = len(hypothesis_id)

      segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
      attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

      token_ids.append(torch.tensor(pair_token_ids))
      seg_ids.append(segment_ids)
      mask_ids.append(attention_mask_ids)
      y.append(self.label_dict[label])
    
    token_ids = pad_sequence(token_ids, batch_first=True)
    mask_ids = pad_sequence(mask_ids, batch_first=True)
    seg_ids = pad_sequence(seg_ids, batch_first=True)
    y = torch.tensor(y)
    dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
    print(len(dataset))
    return dataset

  def get_data_loaders(self, batch_size=32, shuffle=True):
    train_loader = DataLoader(
      self.train_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    val_loader = DataLoader(
      self.val_data,
      shuffle=shuffle,
      batch_size=batch_size
    )

    return train_loader, val_loader

In [ ]:
mnli_dataset = MNLIbertbaseduncased(train_df, val_df)

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc7043

[8038, 11493, 14360, 1006, 2141, 3547, 1007, 2003, 1037, 13329, 2072, 19802, 25879, 2923, 3003, 1998, 2280, 16830, 2040, 13010, 1996, 8745, 1997, 13329, 2013, 2119, 2634, 1998, 4501, 1012, 1999, 2233, 12609, 1010, 14360, 2001, 5338, 2005, 1996, 4028, 1997, 2176, 2796, 2250, 2486, 5073, 2076, 2019, 2886, 1999, 2901, 1998, 2003, 2747, 2104, 3979, 1012, 2002, 2003, 2036, 5307, 3979, 2005, 1996, 15071, 1997, 14548, 4886, 19903, 2360, 13089, 1012, 2002, 2003, 2747, 20451, 1999, 14841, 8167, 7173, 1012]
[2002, 2001, 1996, 3472, 1997, 21433, 13329, 7931, 2392, 1010, 2029, 2761, 27721, 4273, 23689, 25451, 5666, 1999, 1996, 13329, 3028, 1012, 14360, 17738, 22392, 4808, 1999, 2807, 1998, 4233, 9379, 4725, 2000, 2272, 2000, 1037, 4093, 2006, 1996, 13329, 4736, 1012, 2220, 2166, 8038, 11493, 14360, 2001, 2141, 2006, 1017, 2258, 3547, 1999, 1996, 19441, 10357, 14736, 17421, 2050, 10246, 1997, 5185, 14346, 1012]
[14360, 2163, 2008, 1010, 2004, 1037, 2402, 2879, 1010, 2002, 2018, 9741, 4808, 3344, 20

In [ ]:
train_loader, val_loader = mnli_dataset.get_data_loaders(batch_size=16)

In [ ]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
model.to(device)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.2",
  "type_vocab_size":

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 109,484,547 trainable parameters


In [ ]:
def multi_acc(y_pred, y_test):
  acc = (torch.log_softmax(y_pred, dim=1).argmax(dim=1) == y_test).sum().float() / float(y_test.size(0))
  return acc

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, correct_bias=False)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
import time

EPOCHS = 5

def train(model, train_loader, val_loader, optimizer):  
  total_step = len(train_loader)

  for epoch in range(EPOCHS):
    start = time.time()
    model.train()
    total_train_loss = 0
    total_train_acc  = 0
    for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(train_loader):
      optimizer.zero_grad()
      pair_token_ids = pair_token_ids.to(device)
      mask_ids = mask_ids.to(device)
      seg_ids = seg_ids.to(device)
      labels = y.to(device)
      # prediction = model(pair_token_ids, mask_ids, seg_ids)
      loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()

      # loss = criterion(prediction, labels)
      acc = multi_acc(prediction, labels)

      loss.backward()
      optimizer.step()
      
      total_train_loss += loss.item()
      total_train_acc  += acc.item()

    train_acc  = total_train_acc/len(train_loader)
    train_loss = total_train_loss/len(train_loader)
    model.eval()
    total_val_acc  = 0
    total_val_loss = 0
    with torch.no_grad():
      for batch_idx, (pair_token_ids, mask_ids, seg_ids, y) in enumerate(val_loader):
        optimizer.zero_grad()
        pair_token_ids = pair_token_ids.to(device)
        mask_ids = mask_ids.to(device)
        seg_ids = seg_ids.to(device)
        labels = y.to(device)

        # prediction = model(pair_token_ids, mask_ids, seg_ids)
        loss, prediction = model(pair_token_ids, 
                             token_type_ids=seg_ids, 
                             attention_mask=mask_ids, 
                             labels=labels).values()
        
        # loss = criterion(prediction, labels)
        acc = multi_acc(prediction, labels)

        total_val_loss += loss.item()
        total_val_acc  += acc.item()

    val_acc  = total_val_acc/len(val_loader)
    val_loss = total_val_loss/len(val_loader)
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)

    print(f'Epoch {epoch+1}: train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} | val_loss: {val_loss:.4f} val_acc: {val_acc:.4f}')
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

In [ ]:
train(model, train_loader, val_loader, optimizer)

Epoch 1: train_loss: 0.4200 train_acc: 0.8766 | val_loss: 0.0594 val_acc: 1.0000
00:00:28.97
Epoch 2: train_loss: 0.1117 train_acc: 0.9647 | val_loss: 0.0031 val_acc: 1.0000
00:00:29.27
Epoch 3: train_loss: 0.0104 train_acc: 1.0000 | val_loss: 0.0012 val_acc: 1.0000
00:00:29.16
Epoch 4: train_loss: 0.0024 train_acc: 1.0000 | val_loss: 0.0008 val_acc: 1.0000
00:00:29.16
Epoch 5: train_loss: 0.0017 train_acc: 1.0000 | val_loss: 0.0007 val_acc: 1.0000
00:00:29.21


In [ ]:
from transformers import pipeline

classifier = pipeline("text-classification", model = "bert-base-uncased")
classifier("This Nandera modi won the electionin year 2021. This is example of Politics domain.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

[{'label': 'LABEL_0', 'score': 0.5285942554473877}]

In [ ]:
#pip install datasets

In [ ]:
##modification

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

In [ ]:
#import tensorflow as tf
#train_ds = tf.data.Dataset.from_tensor_slices(dict(train_df))
#val_ds = tf.data.Dataset.from_tensor_slices(dict(val_df))

In [ ]:
MODEL="bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(MODEL)

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc7043

In [ ]:
#training_args = TrainingArguments(
#    output_dir='./results',                   # output directory
#    num_train_epochs=EPOCHS,                  # total number of training epochs
#    per_device_train_batch_size=32,           # batch size per device during training
#    per_device_eval_batch_size=32,            # batch size for evaluation
#    warmup_steps=100,                         # number of warmup steps for learning rate scheduler
#    weight_decay=0.01,                        # strength of weight decay
#    logging_dir='./logs',                     # directory for storing logs
#    logging_steps=10,                         # when to print log
#    load_best_model_at_end=True,              # load or not best model at the end
#)

#num_labels = len(set(train_df["label"]))
#model = AlbertForSequenceClassification.from_pretrained(model, num_labels=num_labels)

In [ ]:
checkpoint = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(checkpoint)
def tokenize1_function(train_df):
    return tokenizer(train_df["Premise"], train_df["Hypothesis"], truncation=True,max_length=512)
def tokenize2_function(val_df):
    return tokenizer(val_df["Premise"], val_df["Hypothesis"], truncation=True,max_length=512)
train_df1= tokenize1_function
val_df1= tokenize2_function

loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc7043

In [ ]:
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MyDataset(train_df1, train_df['label'])
val_dataset = MyDataset(val_df1, val_df['label'])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import EarlyStoppingCallback

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
#data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=train_df1,
    eval_dataset=val_df1,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)



ValueError: ignored

In [ ]:
trainer.train()

In [ ]:
import numpy as np
preds = np.argmax(classifier, axis=-1)
label = np.argmax(label)

In [ ]:
from datasets import load_metric

m1 = load_metric("f1")
z1=m1.compute(predictions=preds, references=label_ids)
print(z1)
m2 = load_metric("recall")
z2=m2.compute(predictions=preds, references=label)
print(z2)
m3 = load_metric("accuracy")
z3=m3.compute(predictions=preds, references=label)
print(z3)
m4 = load_metric("matthews_correlation")
z4=m4.compute(predictions=preds, references=label)
print(z4)
m5 = load_metric("pearsonr")
z5=m5.compute(predictions=preds, references=label)
print(z5)
m6 = load_metric("precision")
z6=m6.compute(predictions=preds, references=label)
print(z6)
m7 = load_metric("spearmanr")
z7=m7.compute(predictions=preds, references=label)
print(z7)
m8 = load_metric("xnli")
z8=m8.compute(predictions=preds, references=label)
print(z8)

TypeError: ignored